# Polar Verity Sense PPI stream
3.3.2022, Sakari Lukkarinen<br>
Metropolia University of Applied Sciences<br>

- [Polar Device Stream ECG](https://github.com/pareeknikhil/biofeedback/tree/master/Polar%20Device%20Data%20Stream/ECG)
- [Bleak documentation](https://bleak.readthedocs.io/en/latest)
- [Polar SDK](https://www.polar.com/en/developers/sdk)
    - [Technical documentation](https://github.com/polarofficial/polar-ble-sdk/tree/master/technical_documentation)
    - [Polar Verity Sense sykesensori](https://www.polar.com/fi/tuotteet/lisatarvikkeet/polar-verity-sense)


In [ ]:
## run if the packages are missing ##
#!pip install bleak
#!pip install nest_asyncio

In [ ]:
%matplotlib inline
import os
import sys
import time
import math
import asyncio
import nest_asyncio
import signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display, clear_output
import seaborn as sns
from bleak import BleakClient
from bleak.uuids import uuid16_dict

sns.set_theme()

nest_asyncio.apply()

In [ ]:
## UUID for model number
MODEL_NBR_UUID = "00002a24-0000-1000-8000-00805f9b34fb"

## UUID for manufacturer name
MANUFACTURER_NAME_UUID = "00002a29-0000-1000-8000-00805f9b34fb"

## UUID for battery level
BATTERY_LEVEL_UUID = "00002a19-0000-1000-8000-00805f9b34fb"

## UUID for connection establsihment with device
PMD_SERVICE = "FB005C80-02E7-F387-1CAD-8ACD2D8DF0C8"

## UUID for Request of stream settings 
PMD_CONTROL = "FB005C81-02E7-F387-1CAD-8ACD2D8DF0C8"

## UUID for Request of start stream
PMD_DATA = "FB005C82-02E7-F387-1CAD-8ACD2D8DF0C8"

## UUID for Polar devices
ECG_WRITE = bytearray([0x02, 0x00, 0x00, 0x01, 0x82, 0x00, 0x01, 0x01, 0x0E, 0x00])
START_PPG_STREAM = bytearray([0x02, 0x01, 0x00, 0x01, 0xC8, 0x00, 0x01, 0x01, 0x10, 0x00, 0x02, 0x01, 0x08, 0x00])
START_PPI_STREAM = bytearray([0x02, 0x03])
STOP_PPI_STREAM = bytearray([0x03, 0x03])

## For Polar H10  sampling frequency ##
ECG_SAMPLING_FREQ = 130

In [ ]:
def convert_to_unsigned_long(data, offset, length):
    return int.from_bytes(
        bytearray(data[offset : offset + length]), byteorder="little", signed=False,
    )

def convert_array_to_signed_int(data, offset, length):
    return int.from_bytes(
        bytearray(data[offset : offset + length]), byteorder="little", signed=True,
    )

# Plot and save PPI data
def plot_and_save_data(time_started):
    pd.set_option('precision', 0)
    
    # Convert to Pandas Series, take only the tail of the data and reset index
    ppi = pd.Series(ppi_session_data).tail(RECORD_LENGTH).reset_index(drop = True)
    
    # Plot the data
    plt.figure()
    ppi.plot.line(style = '.-')
    plt.xlabel('Index')
    plt.ylabel('PPI (ms)')
    plt.title('PPI data')
    plt.show()
    
    # Calculate descriptive statistics
    print('Descriptive statistics:')
    print(ppi.describe())
    
    # Save to file, use time_started in naming
    filename = 'Verity Sense Data '+ time_started.strftime('%Y-%m-%d_%H%M.csv')
    ppi.to_csv(filename, header = False, index = False)
    print('\nData saved to file: ', filename)
    
## Bit conversion of the binary stream
def data_conv(sender, data):
    # 0x03 = PPG type
    if data[0] == 0x03:
        # First 8 bytes are for time stamp
        timestamp = convert_to_unsigned_long(data, 1, 8)
        
        # 0x00 = PPI dataframe
        if data[9] == 0x00:
            # Rest are samples in 6 bytes chunks
            samples = data[10:]
            offset = 0
            step = 6
            while offset < len(samples):
                # Heart rate (1 byte)
                hr = samples[offset]
                # Peak-to-peak (2 bytes)
                pp = int.from_bytes(samples[offset+1:offset+3], 'little')
                print("{:4.0f}".format(pp), end = " ")
                ppi_session_data.append(pp)
                
                # Error estimate (2 bytes)
                err_est = int.from_bytes(samples[offset+3:offset+5], 'little')
                # Blocker info (1 byte)
                blocker = samples[offset+5]
                
                offset += step

    
## Aynchronous task to start the data stream for PPI/PPG ##
async def main(address):

    try:
        print("[OPEN] application opened...")
        
        client = BleakClient(address)
        await client.connect()
        print('BLE connection opened.')
        
        model_number = await client.read_gatt_char(MODEL_NBR_UUID)
        print(f'Model Number: {"".join(map(chr, model_number))}')
        
        manufacturer_name = await client.read_gatt_char(MANUFACTURER_NAME_UUID)
        print(f'Manufacturer: {"".join(map(chr, manufacturer_name))}')

        battery_level = await client.read_gatt_char(BATTERY_LEVEL_UUID)
        print(f'Battery Level: {int(battery_level[0])}%')

        att_read = await client.read_gatt_char(PMD_CONTROL)
        if att_read[0] == 0x0F:
            features = att_read[1]
            ECG_SUPPORTED = features&0b000001 != 0
            PPG_SUPPORTED = features&0b000010 != 0
            ACC_SUPPORTED = features&0b000100 != 0
            PPI_SUPPORTED = features&0b001000 != 0
            print('Supported measurement types:') 
            print('    ECG = ', ECG_SUPPORTED)
            print('    PPG = ', PPG_SUPPORTED)
            print('    ACC = ', ACC_SUPPORTED)
            print('    PPI = ', PPI_SUPPORTED) 

        # Start PPI stream
        print('PPI data stream started.')
        await client.write_gatt_char(PMD_CONTROL, START_PPI_STREAM)
        
        ## Data will be collected by data_conv() function everytime
        ## the client notifies that it has data read
        await client.start_notify(PMD_DATA, data_conv)

        ## TypicalLY it takes 30 seconds to get the streaming to start
        print(f"Relax for {WAIT_N_SECONDS} sec...")
        await asyncio.sleep(1)
        time.sleep(WAIT_N_SECONDS)
    
        print("Collecting data...")
        time_started = datetime.now()
    
        print("\nCollecting PPI data for {:} seconds:".format(RECORD_LENGTH))  
        for i in range(RECORD_LENGTH):
            await asyncio.sleep(1)

        ## Stop the stream once data is collected
        await client.write_gatt_char(PMD_CONTROL, STOP_PPI_STREAM)
 
        ## Stop the stream once data is collected ##
        await client.stop_notify(PMD_DATA)
        print("Stopping data streaming...")
 
        plot_and_save_data(time_started)
        
    except Exception as e:
        print(e)
        print(f"Exception: {e=}, {type(e)=}")
        raise
        
    finally:
        await client.disconnect()
        print("[CLOSED] application closed.")

In [ ]:
## This is the device MAC ID, please update with your device ID
ADDRESS = "A0:9E:1A:A9:BB:EB"

# Data will be collected to this list
ppi_session_data = []

# Length of recorded data in seconds
RECORD_LENGTH = 180

# Length of relaxation period in seconds
WAIT_N_SECONDS = 30

## Run the main function asynchronously
asyncio.run(main(ADDRESS))